# Importing Libraries

In [ ]:
!pip install datasist

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Dealing with data and visualization
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline

# For machine learning phase
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, RobustScaler

# for detecting outliers
import datasist as ds


warnings.filterwarnings('ignore')
sns.set(rc={'figure.figsize':[10,10]},font_scale=1.2)
plt.style.use('fivethirtyeight')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Reading the data
df = pd.read_csv('/kaggle/input/housesalesprediction/kc_house_data.csv')
df

> # Data Exploration

In [ ]:
df.info()

**We can deduce that there're no missing values**

In [ ]:
df.describe(include='all')

In [ ]:
# Showing full info about the data
for col in df.columns:
    print(f'For column {col}')
    print(f'Values count:\n{df[col].value_counts()}\n')
    print(f'Unique values: {df[col].unique()}')
    print(f'No. of unique values: {df[col].nunique()}')
    print(f'No. of missing values: {df[col].isnull().sum()}')
    print('*' * 80)
    
print(f'No. of duplicated rows : {df.duplicated().sum()}')

> # Data Wrangling

In [ ]:
# First We should convert some float variables into int
df['price'] = round(df['price']).astype('int64')
df['bathrooms'] = round(df['bathrooms']).astype('int64')
df['floors'] = round(df['floors']).astype('int64')

In [ ]:
# Then we can convert the data column to datetime type
df['date'] = df['date'].apply(lambda x:f'{x[:4]}-{x[4:6]}-{x[6:8]}')

In [ ]:
df['date'] = pd.to_datetime(df['date'],format="%Y-%m-%d")

In [ ]:
df['date']

In [ ]:
df['date'].dt.year.unique()

**So the date column won't be useful so we can drop it**

In [ ]:
df.drop(['date','id'],axis=1,inplace=True)

In [ ]:
# We can rearrange the columns
df = df[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'price']]

In [ ]:
df

> # Data Visualization

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df.corr(),annot=True)

**According to the heatmap, we can drop some unseful columns for better performance in the machine learning phase**

In [ ]:
df = df[['bedrooms', 'bathrooms', 'sqft_living', 'floors',
       'waterfront', 'view', 'condition', 'grade',
       'sqft_basement','price']]

In [ ]:
sns.pairplot(df)

In [ ]:
sns.countplot(df['bedrooms'])

In [ ]:
sns.countplot(df['bathrooms'])

In [ ]:
sns.countplot(df['floors'])

In [ ]:
sns.countplot(df['waterfront'])

In [ ]:
sns.countplot(df['view'])

In [ ]:
sns.countplot(df['condition'])

In [ ]:
sns.countplot(df['grade'])

In [ ]:
sns.kdeplot(df['price'],shade=True)

In [ ]:
sns.boxplot(df['price'])

**There're many outliers in the price column**

In [ ]:
sns.kdeplot(df['bathrooms'],shade=True)

In [ ]:
sns.kdeplot(df['bedrooms'],shade=True)

In [ ]:
sns.boxplot(df['bedrooms'])

In [ ]:
sns.kdeplot(df['sqft_living'],shade=True)

In [ ]:
sns.boxplot(df['sqft_living'])

In [ ]:
sns.kdeplot(df['sqft_basement'],shade=True)

In [ ]:
sns.boxplot(df['sqft_basement'])

**So from the figures above we can deduce that there're many outliers in most of columns**

> # Machine Learning

**We will do 2 trials of machine learning on the data, first one is without removing the outliers and the second one after removing them**

In [ ]:
x = df.drop('price',axis=1)
y = df['price']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=42,test_size=0.25)

In [ ]:
# We are using RobustScaler to reduce the effect of outliers
scaler = RobustScaler()

In [ ]:
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
models = {'LR':LinearRegression(),
         'DTR':DecisionTreeRegressor(),
         'RFR':RandomForestRegressor(),
         'XGR':XGBRegressor()}

**Before removing the outliers**

In [ ]:
for name,model in models.items():
    print(f'For model {name}')
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    print(f'Testing score = {model.score(x_test,y_test)}')
    print(f'Trainig score = {model.score(x_train,y_train)}')
    print(f'R2 score = {r2_score(y_test,y_pred)}')
    print(f'RMSE = {np.sqrt(mean_squared_error(y_pred,y_test))}')
    print('*' * 50)

**We can deduce that XGR is the best model**

**The second trial is after removing the outliers**

In [ ]:
outliers = ds.structdata.detect_outliers(df,0,df.columns)

In [ ]:
df = df[~df.index.isin(outliers)]

In [ ]:
df

In [ ]:
x = df.drop('price',axis=1)
y = df['price']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=42,test_size=0.25)

In [ ]:
# We can use standardscaler as the outliers don't exist now
scaler = StandardScaler()

In [ ]:
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
for name,model in models.items():
    print(f'For model {name}')
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    print(f'Testing score = {model.score(x_test,y_test)}')
    print(f'Trainig score = {model.score(x_train,y_train)}')
    print(f'R2 score = {r2_score(y_test,y_pred)}')
    print(f'RMSE = {np.sqrt(mean_squared_error(y_pred,y_test))}')
    print('*' * 50)

**Removing the outliers wasn't efficient enough so it's better to leave the outliers without removing them**